In [128]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

In [129]:
pd.set_option('display.max_rows', None)
df = pd.read_csv('champs-stats.csv')
df['Abilities'] = df['Abilities'].fillna('')
df.head()

,Name,Class(es),Legacy,Lane,Resource,Adaptive,Difficulty,Abilities
0,Aatrox,['Juggernaut'],"['Fighter', 'Tank']",['Top'],Manaless,Physical,2,"Innate: Periodically, Aatrox empowers his next..."
1,Ahri,['Burst'],"['Mage', 'Assassin']",['Middle'],Mana,Magic,2,Innate: Ahri generates a stack of Essence Frag...
2,Akali,['Assassin'],['Assassin'],"['Top', 'Middle']",Energy,Physical,2,Innate: When Akali damages an enemy champion ...
3,Akshan,"['Marksman', 'Assassin']","['Marksman', 'Assassin']",['Middle'],Mana,Physical,3,"Innate: Whenever Akshan uses a basic attack, h..."
4,Alistar,['Vanguard'],"['Tank', 'Support']",['Support'],Mana,Magic,1,Innate: Alistar generates a stack of Triumph f...


In [130]:
#classes, legacy, lane, resource, adaptive = ohe
#difficulty = nominal encoding
#abiities = tfifd

In [131]:
dfml = df.copy()
multi_label_cols = ['Class(es)', 'Legacy', 'Lane']
encoded_frames = []  

for col in multi_label_cols:
    df[col] = df[col].apply(ast.literal_eval)
    mlb = MultiLabelBinarizer()
    
    encoded = pd.DataFrame(
        mlb.fit_transform(df[col]),
        columns=[f"{col}_{cls}" for cls in mlb.classes_],
        index=df.index
    )
    
    encoded_frames.append(encoded)

dfml = df.drop(columns=multi_label_cols).join(encoded_frames)


In [132]:
vectorizer = TfidfVectorizer()
preproc = make_column_transformer(
    (OneHotEncoder(), ['Resource', 'Adaptive']),
    (TfidfVectorizer(), 'Abilities'),
    remainder='passthrough'
)

champ_names = dfml['Name']
X = preproc.fit_transform(dfml.drop(columns=['Name']))

In [133]:
similarity_matrix = cosine_similarity(X)

def recommend_champions(champ_name, top_n=10):
    
    idx = champ_names[champ_names == champ_name].index[0]
    
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_matches = sim_scores[1:top_n+1]
    
    recommendations = [(champ_names[i], score) for i, score in top_matches]
    return pd.DataFrame(recommendations, columns=['Champion', 'Similarity'])

recommend_champions("Neeko", top_n=10)

,Champion,Similarity
0,Lux,0.818977
1,Morgana,0.811265
2,Brand,0.804260
3,Annie,0.802900
4,Orianna,0.783246
5,Karma,0.779578
6,Veigar,0.776273
7,Xerath,0.746976
8,Vel'Koz,0.746689
9,Swain,0.744319


In [134]:
df.head()

,Name,Class(es),Legacy,Lane,Resource,Adaptive,Difficulty,Abilities
0,Aatrox,[Juggernaut],"[Fighter, Tank]",[Top],Manaless,Physical,2,"Innate: Periodically, Aatrox empowers his next..."
1,Ahri,[Burst],"[Mage, Assassin]",[Middle],Mana,Magic,2,Innate: Ahri generates a stack of Essence Frag...
2,Akali,[Assassin],[Assassin],"[Top, Middle]",Energy,Physical,2,Innate: When Akali damages an enemy champion ...
3,Akshan,"[Marksman, Assassin]","[Marksman, Assassin]",[Middle],Mana,Physical,3,"Innate: Whenever Akshan uses a basic attack, h..."
4,Alistar,[Vanguard],"[Tank, Support]",[Support],Mana,Magic,1,Innate: Alistar generates a stack of Triumph f...


In [139]:
similarity_matrix = cosine_similarity(X)

def recommend_champions(champ_name, top_n=10):
    
    idx = champ_names[champ_names == champ_name].index[0]
    
    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    top_matches = sim_scores[1:top_n+1]
    
    recommendations = [(champ_names[i], score) for i, score in top_matches]
    return pd.DataFrame(recommendations, columns=['Champion', 'Similarity'])

recommend_champions("Lux", top_n=5)

,Champion,Similarity
0,Vel'Koz,0.841368
1,Xerath,0.840264
2,Neeko,0.818977
3,Brand,0.805293
4,Annie,0.801430
